In [234]:
%load_ext autoreload
%autoreload 2

import gurobipy as gp
import itertools
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [276]:
nNodes = 10
nodes = ["n"+str(i+1) for i in range(nNodes)]
# edges = [("n1","n2"),("n1","n3"),("n2","n4"),("n2","n3"),("n2","n5")] + \
#     [("n"+str(i+3), "n"+str(i-1)) for i in range(5, nNodes-4) if i >=0] + \
#     [("n"+str(i), "n"+str(i+6)) for i in range(nNodes-7) if i >= 0] 
edges = [("n"+str(i+1), "n"+str(j+1)) for i in range(nNodes) for j in range(i) if np.random.randn()>0.6]
nAmbul = nNodes//3
TimePeriod = 5
rho  = 0.33

In [277]:
def c_ij(col, j):
    global edges
    global nodes
    global rho

    jc = nodes.index(j)
    total = 0
    
    total = total + (1 if col[jc]==1 else 0)

    neighbors = [n1 for n1 in nodes if (n1, j) in edges] \
        +[n1 for n1 in nodes if (j, n1) in edges] 
    total = total + len([n1 for n1 in neighbors if col[nodes.index(n1)] == 1])*rho
    return total

In [278]:
columns = [tuple([1 for i in range(nAmbul*0)]+[0 for _ in range(nNodes-0*nAmbul)])]
print(columns)

[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)]


In [279]:
iteration = 1
while(1):
    model = gp.Model("MasterLP")
    colVars = model.addVars(columns, vtype=gp.GRB.CONTINUOUS)
    phiVar = model.addVar(obj=1.0, name = "phi")

    model.setObjective(phiVar, gp.GRB.MAXIMIZE)

    model.addConstr(colVars.sum() == TimePeriod)

    constrs = model.addConstrs(gp.quicksum(
        colVars[col]*c_ij(col, j) for col in columns
                                ) >= phiVar for j in nodes)

    model.params.LogToConsole = False
    model.optimize()
    print("Iteration: "+str(iteration)+" Master Objective: ",model.objVal)
    iteration = iteration+1
    ###
    duals = {nn:constrs[nn].getAttr('pi') for nn in nodes}
    # print(duals)

    # Pricing problem
    pricing = gp.Model("pricing")
    cVar = pricing.addVars(nodes, vtype = gp.GRB.CONTINUOUS, obj = duals)
    locVar = pricing.addVars(nodes, vtype = gp.GRB.BINARY)
    for nn in nodes:
        neighbors = [n1 for n1 in nodes if (n1, nn) in edges] \
            +[n1 for n1 in nodes if (nn, n1) in edges] 
        pricing.addConstr(cVar[nn] == locVar[nn] + rho*gp.quicksum(locVar[n1] for n1 in neighbors))
    pricing.addConstr(locVar.sum() == nAmbul)
    pricing.params.LogToConsole = False
    pricing.optimize()
    ###
    colnew = tuple([int(locVar[nn].X) for nn in nodes])
    if colnew in columns:
        print("Complete!!!")
        break
    columns = columns + [colnew]

Iteration: 1 Master Objective:  -0.0
Iteration: 2 Master Objective:  -0.0
Iteration: 3 Master Objective:  -0.0
Iteration: 4 Master Objective:  0.9939759036144579
Iteration: 5 Master Objective:  1.2406015037593985
Iteration: 6 Master Objective:  1.7597347141438502
Iteration: 7 Master Objective:  1.9166168955865597
Iteration: 8 Master Objective:  2.0222991393244794
Iteration: 9 Master Objective:  2.1716400456242333
Iteration: 10 Master Objective:  2.190390266768775
Iteration: 11 Master Objective:  2.192906167215529
Iteration: 12 Master Objective:  2.192906167215529
Complete!!!


In [280]:
for cc in columns:
    colVars[cc].setAttr('vType', 'I')

In [281]:
model.update()
model.optimize()

In [282]:
model.objval

1.9900000000000002

In [294]:
for cc in colVars:
    vv = colVars[cc].X
    if vv > 0.5:
        print(cc, vv, colVars[cc].getAttr("varname"), sep = "---")

(1, 1, 0, 0, 1, 0, 0, 0, 0, 0)---2.0---C3
(0, 0, 0, 1, 0, 0, 0, 1, 1, 0)---1.0---C8
(0, 0, 0, 1, 0, 1, 1, 0, 0, 0)---1.0---C10
(0, 0, 0, 0, 0, 0, 0, 1, 1, 1)---1.0---C11


In [289]:
edges

[('n3', 'n2'),
 ('n4', 'n3'),
 ('n6', 'n2'),
 ('n6', 'n3'),
 ('n7', 'n3'),
 ('n7', 'n4'),
 ('n8', 'n6'),
 ('n10', 'n3'),
 ('n10', 'n4'),
 ('n10', 'n6'),
 ('n10', 'n7')]

# Philippe's code

In [471]:
import networkx
import numpy
from utrecht_scip import *
import math
import gurobipy as gp

In [472]:
EPS = 1e-6

# For efficiency we use the performance target for the Netherlands, i.e.,
# that "95% of all calls should be reached within 15 minutes (900 seconds)".
utrecht = Utrecht()
utrecht.reduce_graph(900)

# Bases are the only vertices that can be allocated ambulances.
bases = utrecht.B
n = len(utrecht.V)

In [473]:
# Replace the seconds by a boolean representing if the zones are close enough
# to cover each other or not.
adj = utrecht.get_binary_adjacency()

# Sufficient coverage required for various population densities.
sufficient = utrecht.get_sufficient_coverage()

In [474]:
# Model parameters
num_ambulances = 20
num_rounds = 3
max_transition = 1 # 0 = no transition allowed, 1 = unlimited transitions
min_coverage = 0.0 # 0 = no one needs to be covered, 1 = everyone has to be covered
max_practical_ambulances = 4 # Maximum practical number of ambulances in a zone (doesn't seem to make much of a difference). This is 4 because a base with 4 ambulances can sufficiently cover any zone no matter what its population density

In [481]:
# Find one feasible configuration for the initial column.
def initial_feasible_configuration():
    """Returns one feasible allocation, along with its coverage."""
    m = gp.Model('M')
    x = [m.addVar(lb=0, ub=max_practical_ambulances, vtype='I', name='x'+str(i)) for i in range(len(bases))]

    # Constraint: Don't use more than the available number of ambulances.
    m.addConstr(gp.quicksum(x) <= num_ambulances)

    # Constraint: The configuration must sufficiently cover 95% of the zones.
    c = [m.addVar(vtype='B', name='c'+str(i)) for i in range(n)]
    for i in range(n):
        m.addGenConstrIndicator(c[i], 1, gp.quicksum(x[j] * adj[bases[j], i] for j in range(len(bases))) >= sufficient[i])
    m.addConstr(gp.quicksum(c) >= math.ceil(min_coverage*n))
    m.update()
    m.optimize()

    allocation = [m.getVarByName('x'+str(i)).X for i in range(len(x))]
    coverage = utrecht.allocation_coverage(allocation)

    return allocation, coverage

In [486]:
# Initial feasible column.
allocation, coverage = initial_feasible_configuration()

# Keeping track of allocations and coverages.
allocations = [allocation]
coverages = [coverage]

# DEBUG: Optimal solution for 20 ambulances, 95% coverage and 3 rounds (solution = 2)
# DEBUG: Uncomment the following lines to start with the optimal solution
allocations = [
    [0, 1, 0, 2, 1, 2, 0, 0, 4, 0, 2, 4, 0, 2, 2, 0, 0, 0],
               [2, 3, 0, 1, 0, 4, 0, 0, 1, 1, 3, 0, 0, 1, 1, 3, 0, 0],
               [2, 1, 1, 1, 0, 3, 0, 0, 2, 0, 0, 4, 2, 0, 3, 0, 1, 0]
              ]
coverages = [utrecht.allocation_coverage(x) for x in allocations]

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 2 rows, 235 columns and 235 nonzeros
Model fingerprint: 0xfd536673
Model has 217 general constraints
Variable types: 0 continuous, 235 integer (217 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 4e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 32 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [483]:
initCvals = []
for i in range(n):
    temp1 = sum([adj[i, bases[j]]*allocations[1][j] for j in range(len(bases))])
    temp2 = 1 if temp1 >= sufficient[i] - EPS else 0
    initCvals.append(temp2)
#     c_vars[i].start = temp2

IndexError: list index out of range

In [484]:
# print(initCvals)
cov = coverages[1]
print([(i,j,initCvals[i]) for i,j in enumerate(cov) if initCvals[i] != j])

IndexError: list index out of range

In [487]:
while True:

    ##### MASTER PROBLEM
    mp = gp.Model('MP')
    mp.Params.OutputFlag = 0

    # Objective.
    phi = mp.addVar(name='phi', vtype='C')
    mp.setObjective(phi, gp.GRB.MAXIMIZE)

    x_vars = [mp.addVar(vtype='C') for _ in allocations]

    # The n constraints constraining the value of phi.
    phi_cons = []
    for i in range(n):
        phi_cons.append(mp.addConstr(gp.quicksum(coverages[j][i]*x_vars[j] for j in range(len(allocations))) - phi >= 0, name='phi'+str(i)))

    # Time horizon constraint: We must use T configurations.
    time_cons = mp.addConstr(gp.quicksum(x_vars) == num_rounds, name='time')
    
    mp.update()
    mp.optimize()
    print('==========>>> MP objective:', mp.getObjective().getValue())
    duals = [mp.getConstrByName(c).Pi for c in ['phi'+str(i) for i in range(n)]]
#     print('==========>>> Duals:', [(j,i) for j,i in enumerate(duals) if i != 0])
    mu = mp.getConstrByName('time').Pi
#     print('==========>>> mu:', mu)

    ##### PRICING PROBLEM
    
    pp = gp.Model('PP')
    pp.Params.OutputFlag = 0
    c_vars = [pp.addVar(vtype=gp.GRB.BINARY, name='c'+str(i)) for i in range(n)]
    u_vars = pp.addVars([i for i in range(len(bases))],vtype=gp.GRB.INTEGER, lb=0, ub=max_practical_ambulances, name='u') 

    # Big M constraints (link c and u).
    bigM = num_ambulances*2
    for i in range(n):
        pp.addGenConstrIndicator(c_vars[i], 1, gp.quicksum(adj[bases[j], i]*u_vars[j] for j in range(len(bases) ) ) >= sufficient[i])
        pp.addGenConstrIndicator(c_vars[i], 0, gp.quicksum(adj[bases[j], i]*u_vars[j] for j in range(len(bases) ) ) <= sufficient[i]-1)
#         pp.addConstr(gp.quicksum(adj[i, bases[j]]*u_vars[j] for j in range(len(bases))) - sufficient[i] + bigM*(1-c_vars[i]) >= 0)
#         pp.addConstr(gp.quicksum(adj[i, bases[j]]*u_vars[j] for j in range(len(bases))) - sufficient[i] + 1 -bigM*c_vars[i] <= 0)

    # Constraint: Don't use more than the available number of ambulances.
    pp.addConstr(u_vars.sum() <= num_ambulances)
    
    # Constraint: The configuration must sufficiently cover 95% of the zones.
#     pp.addConstr(gp.quicksum(c_vars) >= math.ceil(min_coverage*n))

    # Initial point
#     for i in range(len(bases)):
#         u_vars[i].start = allocations[1][i]
        
#     initCvals = []
#     for i in range(n):
#         temp1 = sum([adj[bases[j], i]*allocations[1][j] for j in range(len(bases))])
#         temp2 = 1 if temp1 >= sufficient[i] - EPS else 0
#         initCvals.append(temp2)
#         c_vars[i].start = temp2
        
#     print("*** Objective of starting point ***:", -mu - sum(initCvals[i]*duals[i] for i in range(n)))
    
    # Objective
    pp.setObjective(gp.quicksum(c_vars[i]*duals[i] for i in range(n)), gp.GRB.MINIMIZE)
    pp.update()
    pp.optimize()
#     print('==========>>> PP objective:', pp.getObjective().getValue())
    
    violation =  -(pp.getObjective().getValue() + mu)
    print('=============>>> Dual constraint violation: ',violation)
    

    # Get the newly generated allocation and coverage
    cov = [int(pp.getVarByName('c'+str(i)).X) for i in range(n)]
    alloc = [int(u_vars[i].X) for i in range(len(bases))]
    if alloc in allocations:
        print('==========>>> Generation of an already existing column. Exiting.')
        break
#     print('==========>>> Adding new column')
    coverages.append(cov)
    allocations.append(alloc)
    print()

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333326

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333326

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333326

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333326

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333326

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333326

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333337

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333326

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333337

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  0.33333333333333337

==========>>> MP obj

=============>>> Dual constraint violation:  0.1679389312977101

==========>>> MP objective: 2.391472868217054
=============>>> Dual constraint violation:  0.11208010335917407

==========>>> MP objective: 2.3953846153846157
=============>>> Dual constraint violation:  0.09794871794871807

==========>>> MP objective: 2.4263322884012535
=============>>> Dual constraint violation:  0.14420062695924762

==========>>> MP objective: 2.4375
=============>>> Dual constraint violation:  0.06249999999999989

==========>>> MP objective: 2.4375000000000004
=============>>> Dual constraint violation:  0.1406250000000001

==========>>> MP objective: 2.4374999999999996
=============>>> Dual constraint violation:  0.0625

==========>>> MP objective: 2.4374999999999996
=============>>> Dual constraint violation:  0.06250000000000022

==========>>> MP objective: 2.4374999999999996
=============>>> Dual constraint violation:  0.109375

==========>>> MP objective: 2.4375
=============>>> Dual constraint v

=============>>> Dual constraint violation:  0.024390243902439157

==========>>> MP objective: 2.4878048780487805
=============>>> Dual constraint violation:  0.024390243902439157

==========>>> MP objective: 2.4878048780487805
=============>>> Dual constraint violation:  0.024390243902439046

==========>>> MP objective: 2.4893617021276593
=============>>> Dual constraint violation:  0.04255319148936065

==========>>> MP objective: 2.4893617021276597
=============>>> Dual constraint violation:  0.06382978723404242

==========>>> MP objective: 2.4893617021276593
=============>>> Dual constraint violation:  0.03723404255319185

==========>>> MP objective: 2.4893617021276597
=============>>> Dual constraint violation:  0.0340425531914893

==========>>> MP objective: 2.4893617021276593
=============>>> Dual constraint violation:  0.031914893617020934

==========>>> MP objective: 2.5
=============>>> Dual constraint violation:  -0.0

==========>>> MP objective: 2.5
=============>>> Dual con

In [396]:
adj[4,4]

1

## Debugging

In [388]:
ppd = gp.Model('PP')

c_varsd = [ppd.addVar(vtype='B', name='c'+str(i)) for i in range(n)]
u_varsd = [ppd.addVar(vtype='I', lb=0, ub=max_practical_ambulances, name='u'+str(i)) for i in range(len(bases))]

# Big M constraints (link c and u).
bigM = num_ambulances*2
for i in range(n):
    ppd.addConstr(gp.quicksum(adj[i, bases[j]]*u_varsd[j] for j in range(len(bases))) - sufficient[i] + bigM*(1-c_varsd[i]) >= 0)
    ppd.addConstr(gp.quicksum(adj[i, bases[j]]*u_varsd[j] for j in range(len(bases))) - sufficient[i] + 1 -bigM*c_varsd[i] <= 0)

# Constraint: Don't use more than the available number of ambulances.
ppd.addConstr(gp.quicksum(u_varsd) <= num_ambulances)

# Constraint: The configuration must sufficiently cover 95% of the zones.
ppd.addConstr(gp.quicksum(c_varsd) >= math.ceil(min_coverage*n))

# Objective
ppd.setObjective(gp.quicksum(c_varsd[i]*duals[i] for i in range(n)), gp.GRB.MINIMIZE)
ppd.update()
# ppd.optimize()

In [389]:


allocations[1]

[2, 1, 1, 1, 0, 3, 0, 0, 2, 0, 0, 4, 2, 0, 3, 0, 1, 0]

In [390]:
ppd.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 436 rows, 235 columns and 2767 nonzeros
Model fingerprint: 0x32a8da82
Variable types: 0 continuous, 235 integer (217 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e-01, 5e-01]
  Bounds range     [1e+00, 4e+00]
  RHS range        [1e+00, 4e+01]

User MIP start produced solution with objective -0.5 (0.00s)
Loaded user MIP start with objective -0.5

Presolve removed 39 rows and 19 columns
Presolve time: 0.01s
Presolved: 397 rows, 216 columns, 2382 nonzeros
Variable types: 0 continuous, 216 integer (198 binary)

Root relaxation: objective -1.000000e+00, 71 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      -1.0000000   -1.00000  0.00%     -    0s

Explored 0 nodes (71 simplex iterations) in 0.02 seconds
Thread coun

# Small example

In [429]:
bases = [0, 1, 2, 3, 4] # All zones are bases
n = len(bases)

adj = [[1, 1, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 0, 1]]
adj = numpy.matrix(adj)


sufficient = [2, 2, 2, 2, 2] # Each zone needs to be covered by 2 ambulances. This means that for min_coverage=0.4, all columns where 2 ambulances are allocated to the same node are valid, but also that some other columns where ambulances are allocated to 2 different nodes may also be valid in some cases

# Model parameters
num_ambulances = 2
num_rounds = 3
max_transition = 1 # 0 = no transition allowed, 1 = unlimited transitions
min_coverage = 0.4 # 0 = no one needs to be covered, 1 = everyone has to be covered
max_practical_ambulances = 2 # Maximum practical number of ambulances in a zone (doesn't seem to make much of a difference). This is 4 because a base with 4 ambulances can sufficiently cover any zone no matter what its population density



In [446]:

# Initial column
allocation = [2, 0, 0, 0, 0]
coverage = [2, 2, 0, 0, 0]

# allocation = [0, 0, 2, 0, 0]
# coverage = [0, 2, 2, 2, 2]

# allocation = [1, 0, 0, 0, 1]
# coverage = [1, 1, 1, 0, 1]

# Keeping track of allocations and coverages.
allocations = [allocation]
coverages = [coverage]

In [447]:

while True:

    ##### MASTER PROBLEM
    mp = gp.Model('MP')
    mp.Params.OutputFlag = 0

    # Objective.
    phi = mp.addVar(name='phi', vtype='C')
    mp.setObjective(phi, gp.GRB.MAXIMIZE)

    x_vars = [mp.addVar(vtype='C') for _ in allocations]

    # The n constraints constraining the value of phi.
    phi_cons = []
    for i in range(n):
        phi_cons.append(mp.addConstr(gp.quicksum(coverages[j][i]*x_vars[j] for j in range(len(allocations))) - phi >= 0, name='phi'+str(i)))

    # Time horizon constraint: We must use T configurations.
    time_cons = mp.addConstr(gp.quicksum(x_vars) == num_rounds, name='time')
    
    mp.update()
    mp.optimize()
    print('==========>>> MP objective:', mp.getObjective().getValue())
    duals = [mp.getConstrByName(c).Pi for c in ['phi'+str(i) for i in range(n)]]
#     print('==========>>> Duals:', [i for i in duals if i != 0])
    mu = mp.getConstrByName('time').Pi
#     print('==========>>> mu:', mu)


    ##### PRICING PROBLEM
    
    pp = gp.Model('PP')
    pp.Params.OutputFlag = 0
    c_vars = [pp.addVar(vtype='B', name='c'+str(i)) for i in range(n)]
    u_vars = [pp.addVar(vtype='I', lb=0, ub=max_practical_ambulances, name='u'+str(i)) for i in range(len(bases))]

    # Big M constraints (link c and u).
    bigM = num_ambulances*2
    for i in range(n):
        pp.addConstr(gp.quicksum(adj[i, bases[j]]*u_vars[j] for j in range(len(bases))) - sufficient[i] + bigM*(1-c_vars[i]) >= 0)
        pp.addConstr(gp.quicksum(adj[i, bases[j]]*u_vars[j] for j in range(len(bases))) - sufficient[i] + 1 -bigM*c_vars[i] <= 0)

    # Constraint: Don't use more than the available number of ambulances.
    pp.addConstr(gp.quicksum(u_vars) <= num_ambulances)
    
    # Constraint: The configuration must sufficiently cover 95% of the zones.
    pp.addConstr(gp.quicksum(c_vars) >= math.ceil(min_coverage*n))

    # Objective
    pp.setObjective(gp.quicksum(c_vars[i]*duals[i] for i in range(n)), gp.GRB.MINIMIZE)
    pp.update()
    pp.optimize()
#     print('==========>>> PP objective:', pp.getObjective().getValue())
    violation = - (pp.getObjective().getValue() + mu)
    print('=============>>> Dual constraint violation: ',violation)
#     if violation < EPS:
#         print("Violation of Dual constraint (11b) is very small. ")
    

    # Get the newly generated allocation and coverage
    cov = [int(pp.getVarByName('c'+str(i)).X) for i in range(n)]
    alloc = [int(pp.getVarByName('u'+str(i)).X) for i in range(len(bases))]
    if alloc in allocations:
        print('==========>>> Generation of an already existing column. Exiting.')
        break
#     print('==========>>> Adding new column')
    coverages.append(cov)
    allocations.append(alloc)
    print(alloc)
    print()

==========>>> MP objective: 0.0
=============>>> Dual constraint violation:  1.0
[0, 2, 0, 0, 0]

==========>>> MP objective: 2.0
=============>>> Dual constraint violation:  -0.0
==========>>> Generation of an already existing column. Exiting.


In [448]:
allocations

[[0, 0, 2, 0, 0], [0, 2, 0, 0, 0]]

In [450]:
coverages

[[0, 2, 2, 2, 2], [1, 1, 1, 0, 0]]

In [449]:
x_vars

[<gurobi.Var C1 (value 1.0)>, <gurobi.Var C2 (value 2.0)>]